# Python --> try with stochastic target to emulate real life

In [2]:
import importlib
import numpy as np
from time import time
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
import dna_v3

In [6]:
from dna_v3 import DNA # To get this to work, needed to put 'if name == main' at bottom of dna.py 

In [7]:
importlib.reload(dna_v3)

<module 'dna_v3' from '/home/mjmrose/workspace/sbox-mjmrose/Bids/genAlgo/dna_v3.py'>

# Class Definition

In [34]:
# A class to describe a population of virtual organisms
# In this case, each organism is just an instance of a DNA object

class Population:
    def __init__(self, tgt, mut_rate, pop_sz, fit_exp, eval_perc, mating_pool_retain_perc, replace=True, midpt=False, stoc_tgt=False, verbose=False, debug=False):

        self.population = [] 
        self.mating_pool = [] 
        self.generations = 0
        self.finished = False 
        self.tgt = tgt
        self.mut_rate = mut_rate
        self.perfect_score = 1
        self.best = ""
        self.fitness_sum = 0
        self.fit_exp = fit_exp # Raise fitness to this power to increase (if > 1) prob. of higher fitness members breeding
#       self.num_new_mems = num_new_mems # Num new mems to add to the population or to replace in existing population
#       --> replaced by eval_perc * len(self.population)
        self.eval_perc = eval_perc/100 # % of members of population to evaluate and replace for 2nd+ generation
        self.mating_pool_retain_perc = mating_pool_retain_perc/100 # % of top fitness mems in mating pool to NOT replace
        self.replace_bool = replace # True: replace 'num_new_mems' mems of pop w/ lowest fitness w/ new children // False: new children will be appended to existing population
        self.stoc_tgt_bool = stoc_tgt # True: tgt will change every 100 generations // False: tgt remains stagnant
        self.verbose = verbose
        self.debug = debug
        self.eval_idxs = []
        
        # If not all mems of pop are being evaluated in each generation, can't use fitness for breeding
        if self.eval_perc != 1: self.midpt_bool = False # True: Choose point to split mems being bred // False: probabilistically select gene from one of mems being bred based on fitness
        else: self.midpt_bool = False

        for i in range(pop_sz):
            self.population.append( DNA( len(self.tgt), verbose=self.verbose ) )

            
    def calc_fitness(self):
        '''
        Calculates fitness for every member of the population, exponentiates the fitness and calculates population sum
        '''
        if self.debug: print("start of calc_fitness")
        self.fitness_sum = 0
    
        # For initial population, calculate fitness for every member before generating new members
        if self.generations == 0:
            self.eval_idxs = [i for i in range(len(self.population))]
            for i in self.eval_idxs:
                self.population[i].calc_fitness(self.tgt)
                self.population[i].fitness = self.population[i].fitness**self.fit_exp
                self.fitness_sum += self.population[i].fitness
                
        # For all following generations, evaluate only self.eval_perc % of members
        else:
            if self.eval_perc == 1: self.eval_idxs = [i for i in range(len(self.population))]
                
            # If not evaluating all mems of pop, randomly select eval_perc * len(pop) mems to eval
            else:
                self.eval_idxs = []
                while len(self.eval_idxs) < self.eval_perc * len(self.population):
                    self.eval_idxs.append(np.random.randint(0, len(self.population)))
                    self.eval_idxs = list(set(self.eval_idxs))
                    
            for i in self.eval_idxs:
                self.population[i].calc_fitness(self.tgt)
                self.population[i].fitness = self.population[i].fitness**self.fit_exp
                self.fitness_sum += self.population[i].fitness

            
    def gen_mating_pool(self):
        '''
        Generates mating pool as sorted list of tuples (pop_idx, exponentiated_fitness) w/ highest scoring mems first
        '''
        if self.debug: print("start of gen_mating_pool")
        self.mating_pool = []
    
        for i in range( len(self.eval_idxs) ): 
    
            # Appending (idx, normalized fitness) for each idx in eval_idxs
            self.mating_pool.append( (self.eval_idxs[i], self.population[self.eval_idxs[i]].fitness / max(self.fitness_sum, 1e-4) ) )
    
        # Sorting by fitness score in descending order
        self.mating_pool.sort(reverse=True, key = lambda x : x[1])
        
    
    def pick_mem_from_mating_pool(self):
        '''
        Selects a member (mem) from the mating pool to participate in crossover.

        Steps for selection:
            1). Draw random number b/w 0-1 (val)
            2). Subtract normalized fitness of 1st mem of mating pool (highest fitness) from val
            3). If val is now negative, mem of pop corresponding to first mem of mating pool is selected for crossover.
            4). If val is still positive, move to 2nd mem of mating pool and repeat until val is negative
        '''
        if self.debug: print("Start of pick_mem_from_mating_pool")
        val = np.random.random()
        
        if self.verbose: print(f"val: {val}")
        if self.verbose: print(f"mating pool: {self.mating_pool}")
            
        for i in range( len(self.mating_pool) ):
            val -= self.mating_pool[i][1]
            if val < 0:
                break
        if self.verbose: print(f"idx of mating pool: {self.mating_pool[i][0]}")
        return self.mating_pool[i][0]
    
    def pick_mem_from_population(self):
        '''
        Randomly selects a member (mem) from the population to participate in crossover.
        '''            
        pop_idx = np.random.randint(0, len(self.population))
        return pop_idx
    
    
    def gen_new_pop(self):
        '''
        Generates new members (mems) of population by probabilistically mating existing mems in the mating pool.
        Mems of the mating pool w/ higher fitness are more likely to be selected for mating.
        '''
        if self.debug: print("Start of gen_new_pop")
        children = []
        
        ### Breeding children ###
        # For every member we have evaluated, we need to breed a replacement
        num_children = int( len(self.eval_idxs) * (1 - self.mating_pool_retain_perc) )
        print(f"Generating {num_children} children")
        for i in range( num_children ): 
                
            # Selecting members to be bred
            idx1 = self.pick_mem_from_mating_pool()
            if self.eval_perc == 1: idx2 = self.pick_mem_from_mating_pool()
            else: idx2 = self.pick_mem_from_mating_pool()
            
            #THIS WAS NEW --> would want with low mutation rate
            # Sampling a random (top 10 fitness) member if idx1 == idx2, w/ 50% probability
            if (idx1 == idx2) & (np.random.random() > 0.5): 
                rand_hi_idx = np.random.randint( min(10, len(self.mating_pool) ) )
                idx2 = self.mating_pool[rand_hi_idx][0]
                if self.verbose: print("idx1 == idx2 dealt with")
            
            partnerA = self.population[idx1]
            partnerB = self.population[idx2]
            
            if self.verbose: print(f"PartnerA: {partnerA.get_phrase()}")
            if self.verbose: print(f"PartnerB: {partnerB.get_phrase()}")
    
            child = partnerA.crossover(partnerB, midpt_bool=self.midpt_bool)
            child.mutate(self.mut_rate)
            if self.verbose: print(f"Child: {child.get_phrase()}")
            children.append(child)

        ### Updating population ###
        if self.replace_bool:    
            if self.eval_perc == 1:
                for i in range(len(children)):
                    replace_idx = self.mating_pool[len(self.mating_pool) - i - 1][0]
                    self.population[replace_idx] = children[i] # Overwrites self.population[replace_idx].fitness w/ 0
            else: # Can use mating pool for replacements if it is not too small
                for i in range(len(children)):
#                     replace_idx = self.eval_idxs[i]
                    replace_idx = self.mating_pool[len(self.mating_pool) - i - 1][0]
                    self.population[replace_idx] = children[i] # Overwrites self.population[replace_idx].fitness w/ 0
        else:
            self.population.extend(children)

        self.generations += 1

    def get_best(self):
        return self.best

    def evaluate(self):
        '''
        Computes the current "most fit" member of the population and whether the perfect score has been achieved.
        '''
        world_record = 0
        idx = 0
        for i in range(len(self.population)): 
            if self.population[i].fitness > world_record:
                idx = i
                world_record = self.population[i].fitness
                
        print(f"World Record: {world_record**(1/self.fit_exp)}")

        self.best = self.population[idx].get_phrase()
        if world_record**(1/self.fit_exp) == self.perfect_score:
            self.finished = True
            
        print(f"Best: {self.get_best()}")
        print(f"Average: {self.get_average_fitness()}")

        # If we found the target phrase, stop
        if self.is_finished():
            print("We did it :)")
            print(f"Result: {self.get_best()}")
            print(f"Num gens: {self.get_generations()}")

            
    def get_new_char(self):
        all_possible_chars = string.printable[:-10] + " "
        idx = np.random.randint(len(all_possible_chars))
        return all_possible_chars[idx]

    def is_finished(self):
        return self.finished

    def get_generations(self):
        return self.generations

    def get_average_fitness(self):
        total = 0
        for i in range( len( self.population ) ):
            total += self.population[i].fitness**(1/self.fit_exp)
        return total / len(self.population)
    
    
    def evolve(self):
    
        # Calculate fitness for each mem of pop, take fitness**fit_exp and calc fitness sum
        self.calc_fitness()
        
        # Compute most fit mem of pop and determine if finished
        self.evaluate()

        # Generate mating pool array by sorting normalized fitness values
        self.gen_mating_pool()

        # Generate new population mems by crossover b/w existing mems of mating pool
        # Either replace existing mems or add new mems to pop
        self.gen_new_pop()

        # Updating target if target declared as stochastic
        if (self.stoc_tgt_bool) & (self.get_generations() % 100 == 0):
            idx = np.random.randint(len(self.tgt))
            new_val = self.get_new_char()
            print(f"\nReplacing {self.tgt[idx]} at idx {idx}: {self.tgt[idx]} w/ {new_val}\n")
            new_tgt = []
            for i in range(len(self.tgt)):
                if i != idx:
                    new_tgt.append(self.tgt[i])
                else:
                    new_tgt.append(new_val)
            self.tgt = ''.join(new_tgt)
            print(f"New self.tgt: {self.tgt}\n")

### Initializing a population

In [51]:
tgt = "To be or not to be."
pop_sz = 500
eval_perc = 50
mating_pool_retain_perc = 10
mut_rate = 0.05
fit_exp = 2
pop = Population(tgt, mut_rate, pop_sz, fit_exp, eval_perc, mating_pool_retain_perc, stoc_tgt = True)

# Testing speed of convergence w/ gene selection based on fitness

In [52]:
start = time.time()
for i in range(2000):
    pop.evolve()
    if pop.is_finished():
        break
print(f"\nTime: {time.time() - start}")

World Record: 0.21052631578947367
Best: Z$yq= o$_P;>Nes Ge>
Average: 0.01010526315789474
Generating 450 children
World Record: 0.2631578947368421
Best: N$yq= o1_n;>Nes !e>
Average: 0.049368421052631596
Generating 225 children
World Record: 0.3157894736842105
Best: Z$o$= or_ns>ues !e>
Average: 0.07589473684210532
Generating 225 children
World Record: 0.3684210526315789
Best: N$y/= o1_n;tvtG Ne>
Average: 0.09515789473684205
Generating 225 children
World Record: 0.3684210526315789
Best: Z$y/= o1_n;tvts ke>
Average: 0.10957894736842104
Generating 225 children
World Record: 0.3684210526315789
Best: Z$y/= o1_n;tvts ke>
Average: 0.12736842105263155
Generating 225 children
World Record: 0.42105263157894735
Best: Z$yqe o$_n tvts ke>
Average: 0.14073684210526313
Generating 225 children
World Record: 0.47368421052631576
Best: Zo -e o F0Vto^o ee>
Average: 0.15347368421052635
Generating 225 children
World Record: 0.5263157894736842
Best: 7Umb* or_n)tNto ce>
Average: 0.16621052631578928
Generating 2

In [11]:
0.33 * 500

165.0

# Debugging

In [ ]:
draw(pop, 5, True)

In [ ]:
for i in range(len(pop.population)):
    print(pop.population[i].fitness, pop.population[i].get_phrase())



In [ ]:
pop.mating_pool

In [ ]:
for i in range(len(pop.population)):
    print(pop.population[i].fitness, pop.population[i].get_phrase())


In [ ]:

pop.mating_pool